In [21]:
import numpy as np
import argparse
import imutils
import cv2

In [22]:
def pyramid(image, scale=1.5, minSize=(30, 30)):
    yield image
    while True:
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        yield image

In [23]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0] - windowSize[1], stepSize):
        for x in range(0, image.shape[1] - windowSize[0], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [24]:
def multi_scale_template_matching(image_path, template_path):
    # load the image and template
    image = cv2.imread(image_path)
    template = cv2.imread(template_path)

    # initialize the pyramid and list of scales
    x = pyramid(image, scale=1.5)
    pyramid1 = tuple(x)

    # loop over the pyramid
    for (i, resized) in enumerate(pyramid1):
        # loop over the sliding window for each layer of the pyramid
        for (x, y, window) in sliding_window(resized, stepSize=10, windowSize=template.shape[:2]):
            # crop the ROI and then perform template matching
            roi = resized[y:y + template.shape[0], x:x + template.shape[1]]
            result = cv2.matchTemplate(roi, template, cv2.TM_CCOEFF)
            (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

            # check to see if the iteration should be visualized
            if True:
                # draw a bounding box around the detected region
                clone = np.dstack([resized, resized, resized])
                cv2.rectangle(clone, (x + maxLoc[0], y + maxLoc[1]),
                    (x + maxLoc[0] + template.shape[1], y + maxLoc[1] + template.shape[0]), (0, 0, 255), 2)
                cv2.imshow("Visualize", clone)
                cv2.waitKey(0)

            # if we have found a new maximum correlation value,
            # then update the bookkeeping variable
            if maxVal > found:
                found = maxVal
                (startX, startY) = (x + maxLoc[0], y + maxLoc[1])

    # draw a bounding box around the detected result and display the image
    clone = np.dstack([image, image, image])
    cv2.rectangle(clone, (startX, startY), (startX + tW, startY + tH), (0, 0, 255), 2)
    cv2.imshow("Image", clone)
    cv2.waitKey(0)

SyntaxError: invalid syntax (3343452764.py, line 7)

In [ ]:
multi_scale_template_matching('sample3.jpg', 'board.jpg')